In [1]:
import numpy as np
from sklearn.metrics import roc_curve
from sklearn.neighbors import KNeighborsClassifier
import random
import matplotlib.patheffects as PathEffects
from keras.layers import Input, Conv2D, Lambda, Dense, Flatten,MaxPooling2D, concatenate
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras import backend as K
from keras.optimizers import SGD,Adam
from keras.losses import binary_crossentropy
import os
import pickle
import matplotlib.pyplot as plt
from itertools import permutations
import seaborn as sns
from keras.datasets import mnist
from sklearn.manifold import TSNE
from sklearn.svm import SVC

Using TensorFlow backend.


In [2]:
import sys
import numpy as np
import pandas as pd
import pickle
import os
import matplotlib.pyplot as plt
import cv2
import time

import tensorflow as tf
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model

from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform

from keras.engine.topology import Layer
from keras.regularizers import l2
from keras import backend as K

from sklearn.utils import shuffle

import numpy.random as rng

In [3]:
def imread(path):
    im = cv2.imread(path)
    im = cv2.resize(im, (105,105), interpolation = cv2.INTER_AREA)
    a = np.asarray(im)
    return a

In [4]:
train_folder = "Datasets/101_ObjectCategories/"
val_folder = 'Datasets/101_ObjectCategories/'
save_path = 'Datasets/101_ObjectCategories/data/'

In [5]:
def loadimg(path):
    X=[]
    y=[]
    curr_y=0
    
    for category in os.listdir(path):
        print("loading category: " + category)
        categorypath = path + str(category)
        i=0
        for filename in os.listdir(categorypath):
            image_path = categorypath + "/" + filename
            image = imread(image_path)
            X.append(image)
            y.append(curr_y)
            i=i+1
            if i==30:
                break
        curr_y=curr_y+1
    return X,y
            
    

In [6]:
X,y=loadimg(train_folder)

loading category: accordion
loading category: airplanes
loading category: anchor
loading category: ant
loading category: BACKGROUND_Google
loading category: barrel
loading category: bass
loading category: beaver
loading category: binocular
loading category: bonsai
loading category: brain
loading category: brontosaurus
loading category: buddha
loading category: butterfly
loading category: camera
loading category: cannon
loading category: car_side
loading category: ceiling_fan
loading category: cellphone
loading category: chair
loading category: chandelier
loading category: cougar_body
loading category: cougar_face
loading category: crab
loading category: crayfish
loading category: crocodile
loading category: crocodile_head
loading category: cup
loading category: dalmatian
loading category: dollar_bill
loading category: dolphin
loading category: dragonfly
loading category: electric_guitar
loading category: elephant
loading category: emu
loading category: euphonium
loading category: ewer


In [7]:
def generate_triplet(x,y,testsize=0.3,ap_pairs=10,an_pairs=10):
    data_xy = tuple([x,y])

    trainsize = 1-testsize

    triplet_train_pairs = []
    triplet_test_pairs = []
    for data_class in sorted(set(data_xy[1])):

        same_class_idx = np.where((data_xy[1] == data_class))[0]
        diff_class_idx = np.where(data_xy[1] != data_class)[0]
        A_P_pairs = random.sample(list(permutations(same_class_idx,2)),k=ap_pairs) #Generating Anchor-Positive pairs
        Neg_idx = random.sample(list(diff_class_idx),k=an_pairs)
        

        #train
        A_P_len = len(A_P_pairs)
        Neg_len = len(Neg_idx)
        for ap in A_P_pairs[:int(A_P_len*trainsize)]:
            Anchor = data_xy[0][ap[0]]
            Positive = data_xy[0][ap[1]]
            for n in Neg_idx:
                Negative = data_xy[0][n]
                triplet_train_pairs.append([Anchor,Positive,Negative])               
        #test
        for ap in A_P_pairs[int(A_P_len*trainsize):]:
            Anchor = data_xy[0][ap[0]]
            Positive = data_xy[0][ap[1]]
            for n in Neg_idx:
                Negative = data_xy[0][n]
                triplet_test_pairs.append([Anchor,Positive,Negative])    
                
    return np.array(triplet_train_pairs), np.array(triplet_test_pairs)

In [8]:
y = np.asarray(y)
X = np.asarray(X)

In [22]:
save('X.npy', X)

(3060, 105, 105, 3)


In [9]:
X_train, X_test = generate_triplet(X[0:1500], y[0:1500], ap_pairs=50, an_pairs=50,testsize=0.2)

In [10]:
from numpy import asarray
from numpy import save
save('X_train.npy', X_train)
save('X_test.npy', X_test)

In [11]:
def triplet_loss(y_true, y_pred, alpha = 0.4):
    """
    Implementation of the triplet loss function
    Arguments:
    y_true -- true labels, required when you define a loss in Keras, you don't need it in this function.
    y_pred -- python list containing three objects:
            anchor -- the encodings for the anchor data
            positive -- the encodings for the positive data (similar to anchor)
            negative -- the encodings for the negative data (different from anchor)
    Returns:
    loss -- real number, value of the loss
    """
    print('y_pred.shape = ',y_pred)
    
    total_lenght = y_pred.shape.as_list()[-1]
#     print('total_lenght=',  total_lenght)
#     total_lenght =12
    
    anchor = y_pred[:,0:int(total_lenght*1/3)]
    positive = y_pred[:,int(total_lenght*1/3):int(total_lenght*2/3)]
    negative = y_pred[:,int(total_lenght*2/3):int(total_lenght*3/3)]

    # distance between the anchor and the positive
    pos_dist = K.sum(K.square(anchor-positive),axis=1)

    # distance between the anchor and the negative
    neg_dist = K.sum(K.square(anchor-negative),axis=1)

    # compute loss
    basic_loss = pos_dist-neg_dist+alpha
    loss = K.maximum(basic_loss,0.0)
 
    return loss


In [12]:
def initialize_weights(shape, dtype=None):
    """
        The paper, http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
        suggests to initialize CNN layer weights with mean as 0.0 and standard deviation of 0.01
    """
    return np.random.normal(loc = 0.0, scale = 1e-2, size = shape)


In [13]:
def initialize_bias(shape, dtype=None):
    """
        The paper, http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
        suggests to initialize CNN layer bias with mean as 0.5 and standard deviation of 0.01
    """
    return np.random.normal(loc = 0.5, scale = 1e-2, size = shape)

In [14]:
def create_base_network(input_shape):
    """
    Base network to be shared.
    """
    # Convolutional Neural Network
    model = Sequential()
    model.add(Conv2D(64, (10,10), activation='relu', input_shape=input_shape,
                   kernel_initializer=initialize_weights, kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(128, (7,7), activation='relu',
                     kernel_initializer=initialize_weights,
                     bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(128, (4,4), activation='relu', kernel_initializer=initialize_weights,
                     bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(256, (4,4), activation='relu', kernel_initializer=initialize_weights,
                     bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)))
    model.add(Flatten())
    model.add(Dense(4096, activation='sigmoid',
                   kernel_regularizer=l2(1e-3),
                   kernel_initializer=initialize_weights,bias_initializer=initialize_bias))
    
    return model

In [15]:
adam_optim = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999)

In [16]:
anchor_input = Input((105,105,3, ), name='anchor_input')
positive_input = Input((105,105,3, ), name='positive_input')
negative_input = Input((105,105,3, ), name='negative_input')

# Shared embedding layer for positive and negative items
Shared_DNN = create_base_network([105,105,3,])


encoded_anchor = Shared_DNN(anchor_input)
encoded_positive = Shared_DNN(positive_input)
encoded_negative = Shared_DNN(negative_input)


merged_vector = concatenate([encoded_anchor, encoded_positive, encoded_negative], axis=-1, name='merged_layer')

model = Model(inputs=[anchor_input,positive_input, negative_input], outputs=merged_vector)
model.compile(loss=triplet_loss, optimizer=adam_optim)


y_pred.shape =  Tensor("merged_layer/concat:0", shape=(None, 12288), dtype=float32)


In [17]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
anchor_input (InputLayer)       (None, 105, 105, 3)  0                                            
__________________________________________________________________________________________________
positive_input (InputLayer)     (None, 105, 105, 3)  0                                            
__________________________________________________________________________________________________
negative_input (InputLayer)     (None, 105, 105, 3)  0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 4096)         38960448    anchor_input[0][0]               
                                                                 positive_input[0][0]       

In [ ]:
Anchor = X_train[:,0,:].reshape(-1,105,105,3)
Positive = X_train[:,1,:].reshape(-1,105,105,3)
Negative = X_train[:,2,:].reshape(-1,105,105,3)
Anchor_test = X_test[:,0,:].reshape(-1,105,105,3)
Positive_test = X_test[:,1,:].reshape(-1,105,105,3)
Negative_test = X_test[:,2,:].reshape(-1,105,105,3)

Y_dummy = np.empty((Anchor.shape[0],300))
Y_dummy2 = np.empty((Anchor_test.shape[0],1))

model.fit([Anchor,Positive,Negative],y=Y_dummy,validation_data=([Anchor_test,Positive_test,Negative_test],Y_dummy2), batch_size=512, epochs=500)

Train on 100000 samples, validate on 25000 samples
Epoch 1/500


In [1]:
def save_model(model):
    # serialize model to JSON
    model_json = model.to_json()
    with open("model.json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights("model.h5")
    print("Saved model to disk")


In [2]:
save_model(model)

NameError: name 'model' is not defined